In [ ]:
# 🍖 곱창집 입지 분석 KPI 스코어링 (수정된 모델)
# 03-1_kpi_scoring_fixed.ipynb
#
# 목적: 한식음식점 없는 상권 문제를 해결한 현실적인 곱창집 입지 분석
# 수정사항: 최소 한식음식점 기준 + 경쟁밀도 최적화 곡선 적용
# ================================================================================

# 📚 라이브러리 임포트
# ================================================================
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 🎯 분석 설정
font_family = "Arial, sans-serif"
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("🍖 곱창집 입지 분석 KPI 스코어링 (수정된 모델) 시작!")
print(f"⏰ 시작 시간: {current_time}")
print("🔧 수정사항: 한식음식점 없는 상권 문제 해결")
print("=" * 70)

# 📂 경로 설정 및 데이터 로드
# ================================================================
DATA_PATH = Path("../data")
PROCESSED_PATH = DATA_PATH / "processed"
OUTPUT_PATH = Path("../docs")

# 이전 분석 결과 로드
master_file = OUTPUT_PATH / "gopchang_kpi_analysis.csv"
if master_file.exists():
    master_df = pd.read_csv(master_file, encoding='utf-8-sig')
    print(f"✅ 이전 분석 데이터 로드: {master_df.shape}")
else:
    print("❌ 이전 분석 파일을 찾을 수 없습니다. 먼저 03_kpi_scoring_hybrid.ipynb를 실행해주세요.")

print("=" * 70)

# 🚨 문제 상황 분석
# ================================================================
print("\n🚨 기존 모델 문제점 분석")
print("-" * 30)

# 한식음식점 현황 분석
korean_exists = master_df['한식_점포수'] > 0
korean_no_exists = master_df['한식_점포수'] == 0

print(f"📊 한식음식점 현황:")
print(f"   ✅ 한식음식점 있는 상권: {korean_exists.sum():,}개 ({korean_exists.mean():.1%})")
print(f"   ❌ 한식음식점 없는 상권: {korean_no_exists.sum():,}개 ({korean_no_exists.mean():.1%})")

# 기존 TOP 10 분석
old_top10 = master_df.nlargest(10, '곱창집_적합도_점수')
no_korean_in_top10 = (old_top10['한식_점포수'] == 0).sum()

print(f"\n🔍 기존 TOP 10 분석:")
print(f"   🏆 TOP 10 중 한식음식점 없는 곳: {no_korean_in_top10}개")
print(f"   📋 문제 상권들:")
for i, (idx, row) in enumerate(old_top10.head(5).iterrows(), 1):
    if row['한식_점포수'] == 0:
        print(f"      {i}. {row['상권_코드_명']} - 한식점포 {row['한식_점포수']:.0f}개 ❌")

print("=" * 70)

# 🔧 수정된 필터링 기준 적용
# ================================================================
print("\n🔧 수정된 필터링 기준 적용")
print("-" * 25)

# 필터링 기준 1: 최소 한식음식점 기준
MIN_KOREAN_RESTAURANTS = 1  # 최소 1개 이상
korean_filter = master_df['한식_점포수'] >= MIN_KOREAN_RESTAURANTS

# 필터링 기준 2: 최소 야간 유동인구
MIN_NIGHT_FLOW = 50000  # 5만명 이상
flow_filter = master_df['T1_야간유동인구'] >= MIN_NIGHT_FLOW

# 필터링 기준 3: 최소 한식 매출
MIN_KOREAN_SALES = 1000000  # 100만원 이상
sales_filter = master_df['한식_월매출'] >= MIN_KOREAN_SALES

# 전체 필터 적용
final_filter = korean_filter & flow_filter & sales_filter
filtered_df = master_df[final_filter].copy()

print(f"📊 필터링 결과:")
print(f"   📋 전체 상권: {len(master_df):,}개")
print(f"   ✅ 한식음식점 있는 상권: {korean_filter.sum():,}개")
print(f"   ✅ 야간유동 5만명 이상: {flow_filter.sum():,}개")
print(f"   ✅ 한식매출 100만원 이상: {sales_filter.sum():,}개")
print(f"   🎯 최종 분석 대상: {len(filtered_df):,}개")
print(f"   📈 필터링률: {len(filtered_df)/len(master_df):.1%}")

print("=" * 70)

# 🎯 수정된 KPI 점수 재계산
# ================================================================
print("\n🎯 수정된 KPI 점수 재계산")
print("-" * 20)

def score_normalize_v2(series, reverse=False):
    """개선된 정규화 함수 (0-100점)"""
    if len(series) == 0 or series.std() == 0:
        return pd.Series([50] * len(series), index=series.index)
    
    if reverse:
        scores = 100 - ((series - series.min()) / (series.max() - series.min()) * 100)
    else:
        scores = (series - series.min()) / (series.max() - series.min()) * 100
    
    return scores.fillna(50)

def optimal_competition_score(restaurant_count, optimal_range=(3, 8)):
    """
    최적 경쟁밀도 점수 계산
    - 너무 적으면 (1-2개): 상권 매력도 부족
    - 적절하면 (3-8개): 검증된 상권이지만 과도한 경쟁 아님
    - 너무 많으면 (9개+): 과도한 경쟁
    """
    scores = []
    for count in restaurant_count:
        if count < optimal_range[0]:
            # 너무 적음: 선형 감소
            score = (count / optimal_range[0]) * 80
        elif count <= optimal_range[1]:
            # 최적 구간: 높은 점수
            score = 80 + ((optimal_range[1] - count) / (optimal_range[1] - optimal_range[0])) * 20
        else:
            # 너무 많음: 지수적 감소
            excess = count - optimal_range[1]
            score = max(20, 80 * np.exp(-excess * 0.3))
        scores.append(score)
    
    return np.array(scores)

# 1. 필터링된 데이터에서 점수 재계산
print("1️⃣ T1: 야간 유동인구 점수 재계산")
filtered_df['T1_점수_v2'] = score_normalize_v2(filtered_df['T1_야간유동인구'])
print(f"   📊 평균 점수: {filtered_df['T1_점수_v2'].mean():.1f}점")

print("2️⃣ T2: 한식 매출밀도 점수 재계산")
# 매출밀도 재계산 (0 방지)
filtered_df['T2_한식매출밀도_v2'] = filtered_df['한식_월매출'] / filtered_df['T1_야간유동인구']
filtered_df['T2_점수_v2'] = score_normalize_v2(filtered_df['T2_한식매출밀도_v2'])
print(f"   📊 평균 점수: {filtered_df['T2_점수_v2'].mean():.1f}점")

print("3️⃣ C1: 최적 경쟁밀도 점수 재계산")
filtered_df['C1_점수_v2'] = optimal_competition_score(filtered_df['한식_점포수'])
print(f"   📊 평균 점수: {filtered_df['C1_점수_v2'].mean():.1f}점")
print(f"   📋 최적 경쟁밀도: 3-8개 한식음식점")

print("4️⃣ C2: 상권 활성도 점수 재계산")
filtered_df['C2_점수_v2'] = score_normalize_v2(filtered_df['C2_상권활성도'])
print(f"   📊 평균 점수: {filtered_df['C2_점수_v2'].mean():.1f}점")

print("5️⃣ E1: 주류친화 지수 점수 재계산")
filtered_df['E1_점수_v2'] = score_normalize_v2(filtered_df['E1_주류친화지수'])
print(f"   📊 평균 점수: {filtered_df['E1_점수_v2'].mean():.1f}점")

print("=" * 70)

# 🏆 수정된 최종 점수 계산
# ================================================================
print("\n🏆 수정된 최종 점수 계산")
print("-" * 20)

# 동일한 가중치 적용
weights_v2 = {
    'T1_점수_v2': 0.30,  # 야간 유동인구 (30%)
    'T2_점수_v2': 0.25,  # 한식 매출밀도 (25%)
    'C1_점수_v2': 0.20,  # 최적 경쟁밀도 (20%)
    'C2_점수_v2': 0.15,  # 상권활성도 (15%)
    'E1_점수_v2': 0.10   # 주류친화지수 (10%)
}

filtered_df['곱창집_적합도_v2'] = (
    filtered_df['T1_점수_v2'] * weights_v2['T1_점수_v2'] +
    filtered_df['T2_점수_v2'] * weights_v2['T2_점수_v2'] +
    filtered_df['C1_점수_v2'] * weights_v2['C1_점수_v2'] +
    filtered_df['C2_점수_v2'] * weights_v2['C2_점수_v2'] +
    filtered_df['E1_점수_v2'] * weights_v2['E1_점수_v2']
)

print(f"📊 수정된 최종 적합도 점수:")
print(f"   평균: {filtered_df['곱창집_적합도_v2'].mean():.1f}점")
print(f"   최고: {filtered_df['곱창집_적합도_v2'].max():.1f}점")
print(f"   최저: {filtered_df['곱창집_적합도_v2'].min():.1f}점")
print(f"   표준편차: {filtered_df['곱창집_적합도_v2'].std():.1f}점")

print("=" * 70)

# 🥇 수정된 TOP 30 곱창집 최적 입지
# ================================================================
print("\n🥇 수정된 TOP 30 곱창집 최적 입지")
print("-" * 30)

# TOP 30 추출
top30_v2 = filtered_df.nlargest(30, '곱창집_적합도_v2')[
    ['상권_코드_명', '곱창집_적합도_v2', 'T1_야간유동인구', '한식_월매출', 
     '한식_점포수', 'C2_상권활성도', 'T1_점수_v2', 'T2_점수_v2', 
     'C1_점수_v2', 'C2_점수_v2', 'E1_점수_v2']
].copy()

print("🏆 수정된 곱창집 입지 TOP 30:")
print("-" * 70)
for i, (idx, row) in enumerate(top30_v2.iterrows(), 1):
    print(f"{i:2d}. {row['상권_코드_명']:<30} "
          f"(적합도: {row['곱창집_적합도_v2']:.1f}점)")
    print(f"    🌙 야간유동: {row['T1_야간유동인구']:8,.0f}명 "
          f"💰 한식매출: {row['한식_월매출']/100000000:6.1f}억원 "
          f"🏪 한식점포: {row['한식_점포수']:2.0f}개")
    if i <= 10:  # TOP 10만 상세 표시
        print(f"    📊 점수: T1({row['T1_점수_v2']:.0f}) T2({row['T2_점수_v2']:.0f}) "
              f"C1({row['C1_점수_v2']:.0f}) C2({row['C2_점수_v2']:.0f}) E1({row['E1_점수_v2']:.0f})")
    print()

print("=" * 70)

# 📊 개선 전후 비교 분석
# ================================================================
print("\n📊 개선 전후 비교 분석")
print("-" * 20)

# 기존 TOP 10과 수정된 TOP 10 비교
old_top10_names = set(old_top10.head(10)['상권_코드_명'].tolist())
new_top10_names = set(top30_v2.head(10)['상권_코드_명'].tolist())

print(f"🔄 TOP 10 변화:")
print(f"   📊 기존 모델: {list(old_top10_names)[:5]}...")
print(f"   📊 수정 모델: {list(new_top10_names)[:5]}...")
print(f"   🔄 공통 상권: {len(old_top10_names & new_top10_names)}개")
print(f"   ✨ 새로운 상권: {len(new_top10_names - old_top10_names)}개")

# 한식음식점 현황 비교
print(f"\n🍖 한식음식점 현황 개선:")
old_korean_avg = old_top10.head(10)['한식_점포수'].mean()
new_korean_avg = top30_v2.head(10)['한식_점포수'].mean()
print(f"   📊 기존 TOP 10 평균 한식점포: {old_korean_avg:.1f}개")
print(f"   📊 수정 TOP 10 평균 한식점포: {new_korean_avg:.1f}개")
print(f"   📈 개선 정도: {((new_korean_avg - old_korean_avg) / old_korean_avg * 100):+.1f}%")

print("=" * 70)

# 📊 수정된 시각화
# ================================================================
print("\n📊 수정된 분석 시각화")
print("-" * 15)

# 1. 개선 전후 TOP 10 비교 차트
comparison_data = []
for i in range(10):
    old_row = old_top10.iloc[i]
    new_row = top30_v2.iloc[i]
    comparison_data.append({
        '순위': f"TOP {i+1}",
        '기존모델': old_row['상권_코드_명'],
        '기존점수': old_row['곱창집_적합도_점수'],
        '수정모델': new_row['상권_코드_명'],
        '수정점수': new_row['곱창집_적합도_v2']
    })

comparison_df = pd.DataFrame(comparison_data)

fig_comparison = go.Figure()

fig_comparison.add_trace(go.Bar(
    name='기존 모델',
    x=comparison_df['순위'],
    y=comparison_df['기존점수'],
    text=comparison_df['기존모델'],
    textposition='outside',
    textangle=45,
    marker_color='lightcoral'
))

fig_comparison.add_trace(go.Bar(
    name='수정 모델',
    x=comparison_df['순위'], 
    y=comparison_df['수정점수'],
    text=comparison_df['수정모델'],
    textposition='outside',
    textangle=45,
    marker_color='lightblue'
))

fig_comparison.update_layout(
    title="🔄 개선 전후 TOP 10 비교",
    xaxis_title="순위",
    yaxis_title="적합도 점수",
    barmode='group',
    font=dict(family=font_family, size=12),
    height=600
)

fig_comparison.show()

# 2. 경쟁밀도 최적화 곡선
restaurant_counts = np.arange(1, 16)
optimal_scores = optimal_competition_score(restaurant_counts)

fig_optimal = go.Figure()

fig_optimal.add_trace(go.Scatter(
    x=restaurant_counts,
    y=optimal_scores,
    mode='lines+markers',
    name='최적 경쟁밀도 점수',
    line=dict(color='red', width=3),
    marker=dict(size=8)
))

# 최적 구간 강조
fig_optimal.add_vrect(
    x0=3, x1=8,
    fillcolor="green", opacity=0.2,
    layer="below", line_width=0,
    annotation_text="최적 구간"
)

fig_optimal.update_layout(
    title="🎯 한식음식점 경쟁밀도 최적화 곡선",
    xaxis_title="한식음식점 개수",
    yaxis_title="경쟁밀도 점수",
    font=dict(family=font_family, size=12),
    height=500
)

fig_optimal.show()

# 3. 수정된 TOP 15 적합도 점수
fig_top15_v2 = px.bar(
    top30_v2.head(15),
    x='상권_코드_명',
    y='곱창집_적합도_v2',
    title="🏆 수정된 곱창집 입지 TOP 15",
    color='곱창집_적합도_v2',
    color_continuous_scale='Reds'
)

fig_top15_v2.update_layout(
    xaxis_title="상권명",
    yaxis_title="적합도 점수",
    font=dict(family=font_family, size=12),
    height=500,
    xaxis={'tickangle': 45}
)

fig_top15_v2.show()

print("=" * 70)

# 💾 수정된 결과 저장
# ================================================================
print("\n💾 수정된 분석 결과 저장")
print("-" * 20)

# 전체 결과 저장
output_csv_v2 = OUTPUT_PATH / "gopchang_kpi_analysis_v2.csv"
filtered_df.to_csv(output_csv_v2, index=False, encoding='utf-8-sig')
print(f"✅ 수정된 전체 결과: {output_csv_v2.name}")

# TOP 30 저장
top30_csv_v2 = OUTPUT_PATH / "gopchang_top30_locations_v2.csv"
top30_v2.to_csv(top30_csv_v2, index=False, encoding='utf-8-sig')
print(f"✅ 수정된 TOP 30: {top30_csv_v2.name}")

# 개선 보고서 저장
report_v2 = OUTPUT_PATH / "gopchang_improvement_report.txt"
with open(report_v2, 'w', encoding='utf-8') as f:
    f.write("🍖 곱창집 입지 분석 모델 개선 보고서\n")
    f.write(f"📅 분석일시: {current_time}\n")
    f.write("=" * 50 + "\n\n")
    
    f.write("🚨 기존 모델 문제점:\n")
    f.write("• 한식음식점 없는 상권이 상위권 차지\n")
    f.write("• 경쟁이 없다 = 좋다는 잘못된 논리\n")
    f.write("• 현실성 부족한 추천 결과\n\n")
    
    f.write("🔧 개선 사항:\n")
    f.write("• 최소 한식음식점 1개 이상 필터링\n")
    f.write("• 최적 경쟁밀도 곡선 적용 (3-8개 최적)\n")
    f.write("• 최소 야간유동인구 5만명 기준\n")
    f.write("• 최소 한식매출 100만원 기준\n\n")
    
    f.write("🏆 수정된 TOP 10:\n")
    for i, (idx, row) in enumerate(top30_v2.head(10).iterrows(), 1):
        f.write(f"{i:2d}. {row['상권_코드_명']} ({row['곱창집_적합도_v2']:.1f}점)\n")
    
    f.write(f"\n📈 개선 효과:\n")
    f.write(f"• 분석 대상: {len(master_df):,}개 → {len(filtered_df):,}개\n")
    f.write(f"• 평균 한식점포: {old_korean_avg:.1f}개 → {new_korean_avg:.1f}개\n")
    f.write(f"• 현실성 있는 입지 추천 완성\n")

print(f"✅ 개선 보고서: {report_v2.name}")

print("=" * 70)

# 🎯 최종 완료 요약
# ================================================================
print("\n🎯 곱창집 입지 분석 모델 개선 완료!")
print("-" * 35)

end_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"✅ 완료 시간: {end_time}")

print(f"\n📊 개선 결과 요약:")
print(f"   🔧 필터링 기준: 3개 조건 적용")
print(f"   🎯 최적화 곡선: 경쟁밀도 3-8개 최적")
print(f"   📈 현실성 향상: 한식점포 평균 {new_korean_avg:.1f}개")
print(f"   🏆 신뢰성 있는 TOP 30 완성")

print(f"\n🍖 핵심 개선 인사이트:")
print(f"   • 수정된 1위: {top30_v2.iloc[0]['상권_코드_명']} ({top30_v2.iloc[0]['곱창집_적합도_v2']:.1f}점)")
print(f"   • 한식 생태계 고려: 너무 적지도, 많지도 않은 최적점")
print(f"   • 검증된 상권 우선: 기존 한식음식점 성공 지역")

print(f"\n🚀 추천 다음 단계:")
print(f"   A. 지도 시각화 (수정된 TOP 30)")
print(f"   B. 상권별 세부 분석")
print(f"   C. 최종 추천 보고서")

print("=" * 70)
print("🍖 현실성 있는 곱창집 입지 분석 완료!")
print("🎯 이제 진짜 실용적인 TOP 30 입지를 확보했습니다!")